In [ ]:
# default_exp db

In [ ]:
%reload_ext autoreload
%autoreload 2

# 数据库
> Redis，MongoDB如何选择

为什么要用数据库？因为保存为单独文件的读写性能非常的弱，而且数据要全部加载到内存中才能进行分析，而数据库则具有更好的性能

## Redis
> 

In [ ]:
# export
import requests,json,re,random,sys
from bs4 import BeautifulSoup,Tag,NavigableString

from crawler_from_scratch.utils import *

from concurrent.futures import ThreadPoolExecutor
import pandas as pd


In [ ]:
pd.DataFrame.from

AttributeError: type object 'DataFrame' has no attribute 'redis'

In [ ]:
def gen():
    for i in range(10):
        yield i

In [ ]:
gen().length

AttributeError: 'generator' object has no attribute 'length'

In [ ]:
# hide
url = 'https://www.baidu.com/'
headers={'user-agent':'Mozilla/5.0'}
proxies = {'https': '140.238.15.33:3128'}
res = requests.get(url,proxies=proxies,headers=headers,timeout=5)
res,res.text

(<Response [200]>,
 '<html>\r\n<head>\r\n\t<script>\r\n\t\tlocation.replace(location.href.replace("https://","http://"));\r\n\t</script>\r\n</head>\r\n<body>\r\n\t<noscript><meta http-equiv="refresh" content="0;url=http://www.baidu.com/"></noscript>\r\n</body>\r\n</html>')